In [278]:
from pathlib import Path
from decode_fish.funcs.utils import *
import glob

def make_sweep_str(pfile, pars, group, resume=False, init=None, gpu='gpu_rtx'):
    
    base_string = f'bsub -gpu "num=1" -q {gpu} -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py'   
    
    base_str = base_string + f' +experiment={pfile}'
    name = ''
    
    for k in pars:
        if k!= 'run_name':
            base_str += ' ' + k + f'={pars[k]}'
            name += k.split('.')[-1]+f':{pars[k]}x'
            name = name.replace('[', '')
            name = name.replace(']', '')
            name = name.replace(',', 'x')
    name = name[:-1]
    
    if 'run_name' in pars:
        name=pars['run_name']
    
    if resume:
        base_str += ' training.resume=True '
    if init is not None:
        base_str += f' data_path.model_init={init} '
    
    return base_str + f' run_name={name} output.group={group}' + '\n'
    

def make_named_str(pfile, name, group):
    
    base_str = base_string + f' +experiment={pfile}'
    
    return base_str + f' run_name={name} output.group={group}' + '\n'

In [179]:
# cfg.training.mic.par_grads.channel_facs = False
# cfg.training.mic.par_grads.channel_shifts = False
# cfg.training.mic.par_grads.theta_par = False
# cfg.training.mic.par_grads.psf_vol = False

In [576]:
group = 'sweep_sl_21'
pfile = 'MERFISH_ci_2d_sl21'

variable_col = param_iter()
variable_col.add('training.bs', 20)
variable_col.add('genm.exp_type.int_inf', 'per_channel')
par_prod = variable_col.param_product()

In [300]:
group = 'sweep_mop_5b_psf_opt'
pfile = 'MERFISH_mop_5b_psf'

variable_col = param_iter()
variable_col.add('training.mic.l1_reg', 0.0003)
variable_col.add('training.mic.opt.lr', 3e-5)
variable_col.add('training.mic.par_grads.channel_shifts', True)
variable_col.add('training.mic.par_grads.channel_facs', True)
par_prod = variable_col.param_product()

In [338]:
group = 'sweep_mop_5b'
pfile = 'MERFISH_mop_5c'

variable_col = param_iter()
variable_col.add('genm.microscope.psf_noise', 0.15)
# variable_col.add('genm.microscope.norm', 'sum')
# variable_col.add('training.mic.l1_reg', 0.0003)
variable_col.add('training.mic.par_grads.psf_vol', False)
# variable_col.add('genm.microscope.scale', 300)
par_prod = variable_col.param_product()

In [612]:
group = 'sweep_mop_9b'
pfile = 'MERFISH_mop_9b'

variable_col = param_iter()
variable_col.add('seed',1)

par_prod = variable_col.param_product()

In [660]:
group = 'sweep_mop_11'
pfile = 'MERFISH_mop_11b'

variable_col = param_iter()
# variable_col.add('genm.microscope.pos_noise_xy', 0.2, 0.35, 0.5)
variable_col.add('training.mic.enabled', False)
# variable_col.add('training.start_mic', 3)
par_prod = variable_col.param_product()

In [661]:
# Path.mkdir(model_path/group, exist_ok=True)

with open('run_slurm.sh', 'w') as f:
    for pars in par_prod:  
        exec_str = make_sweep_str(pfile, pars, group, resume=False, gpu='gpu_rtx') #, 
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_rtx -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev2/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=MERFISH_mop_11b training.mic.enabled=False run_name=enabled:False output.group=sweep_mop_11



In [78]:
# sweep = 'sweep_sl_8'

# dirs = glob.glob(f'/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/models/fishcod/MERFISH_starfish/{sweep}/*')
# with open('run_slurm.sh', 'w') as f:
#     for d in dirs:
#         name = d.split('/')[-1]
#         exec_str = f'bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode_fish_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/merfish_eval.py run_name={name} sweep_name={sweep}'  + '\n'
#         print(exec_str)
#         f.write(exec_str)

In [120]:
group = 'sep_models_ch0_3'
pfile = 'CL_dlg1_ch0_f1'

variable_col = param_iter()
variable_col.add('data_path.image_path', 
                 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/raw_data_for_codetection/*p1*.tiff',
                 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/raw_data_for_codetection/*p2*.tiff',
                 '/groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/datasets/CodFish/raw_data_for_codetection/*p3*.tiff')

par_prod = variable_col.param_product()

In [40]:
group = 'sweep_fig9'
pfile = 'fig_sim_m9'

variable_col = param_iter()
variable_col.add('training.mic.enabled', True, False)
variable_col.add('training.int.enabled', True, False)

par_prod = variable_col.param_product()

In [93]:
group = 'sweep_psf2'
pfile = 'N2_352_f1'

variable_col = param_iter()
variable_col.add('microscope.psf_noise', 0.1, 0.2)
variable_col.add('microscope.double_random', True, False)
variable_col.add('model.f_maps', 32, 42)

par_prod = variable_col.param_product()

In [36]:
from decode_fish.imports import *

psfn_3: psf_noise = 0.5
psfn_4: psf_noise = 0.75
psfn_5: psf_noise = 1.
psfn_6: psf_noise = 1. double random
psfn_7: psf_noise = 2. double random
psfn_8: psf_noise = 3. double random

In [88]:
exps = ['msp300', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1', 'CL_dlg1_ch0', 'CL_dlg1_ch1']
exps = ['CL_dlg1_ch1']
ver = 'f1'
name = 'f1_psfn_5'

with open('run_slurm.sh', 'w') as f:
    for exp in exps:  
        cfg = OmegaConf.load(f'config/experiment/{exp}_{ver}.yaml')
#         exec_str = base_string + f' +experiment={exp}_{ver}  run_name={exp}_{name}' + '\n'
        exec_str = base_string + f' +experiment={exp}_{ver}  run_name={exp}_{name} noise.theta={cfg.noise.theta*1.5} microscope.psf_noise=0.1' + '\n'
        print(exec_str)
        f.write(exec_str)

bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/train.py +experiment=CL_dlg1_ch1_f1  run_name=CL_dlg1_ch1_f1_psfn_5 noise.theta=75.0 microscope.psf_noise=0.1



In [ ]:
# bsub -gpu "num=1" -q gpu_any -o logs/ofish.log -e logs/efish.log /groups/turaga/home/speisera/anaconda3/envs/decode2_dev/bin/python /groups/turaga/home/speisera/Mackebox/Artur/WorkDB/deepstorm/decode_fish/decode_fish/sim_eval.py sweep_name=fig7

In [ ]:
exps = ['msp300', 'N2_352', 'rab11', 'scrib', 'sgg', 'Dlg1']
# exps = [ 'scrib', 'Dlg1']
ver = 'f1'
name = 'f1_none'

In [314]:
import numpy as np
zoe_ps = np.array([81,88,85,79,65,69,67,70,63,80,80,74,69,85,71,80,70,81,80,79,89,84,87,79,80,90,80,78,81,83,76,100,85,95,87,91,76,73,95,88,79,67,90,89,82,87,88,76,81,87,85,78,83,84,84,76,75,88,81,79,87,91,83,91,75,86,80,95,72,73,76,81,82,89,81,86,72])

In [319]:
zoe_ps

array([ 81,  88,  85,  79,  65,  69,  67,  70,  63,  80,  80,  74,  69,
        85,  71,  80,  70,  81,  80,  79,  89,  84,  87,  79,  80,  90,
        80,  78,  81,  83,  76, 100,  85,  95,  87,  91,  76,  73,  95,
        88,  79,  67,  90,  89,  82,  87,  88,  76,  81,  87,  85,  78,
        83,  84,  84,  76,  75,  88,  81,  79,  87,  91,  83,  91,  75,
        86,  80,  95,  72,  73,  76,  81,  82,  89,  81])